In [2]:
%%capture
%%pip install pulp

In [3]:
from pulp import *
import pulp
import pandas as pd
import numpy as np
import random
from builtins import list

In [4]:


# Lemos o ficheiro com os resultados das previsões dos produtos
df_productPredictions = pd.read_excel('../ml/results/results.xlsx')

# Verificamos o dataframe obtido
print(df_productPredictions.head())



  product_id                  model    rmse_sales  rmse_revenue     mae_sales  \
0      P0001       LinearRegression  4.885453e-01  2.578147e+00  4.041303e-01   
1      P0001  RandomForestRegressor  4.541629e+00  2.643378e+01  2.912128e+00   
2      P0001           XGBRegressor  3.921948e+00  2.690934e+01  2.526265e+00   
3      P0002       LinearRegression  1.218277e-15  2.703535e-14  1.016386e-15   
4      P0002  RandomForestRegressor  1.381411e+00  2.095142e+02  3.619048e-01   

    mae_revenue  r2_sales  r2_revenue  periodicity_total_pred_sales  \
0  2.147004e+00  0.999275    0.999468                            20   
1  1.623660e+01  0.937366    0.944062                            32   
2  1.796265e+01  0.953292    0.942031                            32   
3  1.974587e-14  1.000000    1.000000                             1   
4  6.351167e+01  0.380296    0.511473                             0   

   periodicity_total_pred_revenue  
0                          196.86  
1             

In [5]:


# Lemos o ficheiro com as dimensões dos produtos
df_productDimensions = pd.read_csv('../dataset/product_hierarchy.csv')

# Verificamos o dataframe obtido
print(df_productDimensions.head())



  product_id  product_length  product_depth  product_width cluster_id  \
0      P0000             5.0           20.0           12.0        NaN   
1      P0001            13.5           22.0           20.0  cluster_5   
2      P0002            22.0           40.0           22.0  cluster_0   
3      P0004             2.0           13.0            4.0  cluster_3   
4      P0005            16.0           30.0           16.0  cluster_9   

  hierarchy1_id hierarchy2_id hierarchy3_id hierarchy4_id hierarchy5_id  
0           H00         H0004       H000401     H00040105   H0004010534  
1           H01         H0105       H010501     H01050100   H0105010006  
2           H03         H0315       H031508     H03150800   H0315080028  
3           H03         H0314       H031405     H03140500   H0314050003  
4           H03         H0312       H031211     H03121109   H0312110917  


In [6]:


# Adicionamos uma nova coluna com a média de desempenho das previsões de receitas e vendas de cada modelo aplicado a cada produto
df_productPredictions['average_r2'] = (df_productPredictions['r2_sales'] + df_productPredictions['r2_revenue']) / 2

# Agrupamos os dados por produto, escolhendo apenas o modelo com a melhor média de desempenho das previsões, com o intuito de escolhermos os valores mais precisos
df_productPredictions = df_productPredictions.groupby('product_id').apply(lambda x: x.loc[x['average_r2'].idxmax()]).reset_index(level=0, drop=True)

# Verificamos a distribuição dos modelos escolhidos
print(df_productPredictions.model.value_counts().head())



LinearRegression         592
RandomForestRegressor      4
XGBRegressor               3
Name: model, dtype: int64


In [7]:


# Juntamos as dimensões dos produtos
df = pd.merge(df_productPredictions, df_productDimensions, left_on='product_id', right_on='product_id', how='left')

# Adicionamos uma nova coluna com as dimensões cúbicas (i.e. volume)
df['dimensions'] = (df['product_length'] * df['product_depth'] * df['product_width'])

# Removemos as colunas desnecessárias
df = df[['product_id', 'periodicity_total_pred_sales', 'periodicity_total_pred_revenue', 'dimensions']]
df = df.fillna(0)

# Verificamos o dataframe final
print(df.head())



  product_id  periodicity_total_pred_sales  periodicity_total_pred_revenue  \
0      P0001                            20                          196.86   
1      P0002                             1                          230.01   
2      P0004                             1                            0.85   
3      P0005                            12                          499.93   
4      P0006                             1                            0.00   

   dimensions  
0      5940.0  
1     19360.0  
2       104.0  
3      7680.0  
4      1912.5  


In [8]:
#-------------------------------User Input-------------------------------------

# Perguntamos pelos valores pretendidos para as restrições

minAmountAllProducts = int(input("Por favor insira a quantidade mínima global de produtos: "))
maxAmountAllProducts = int(input("Por favor insira a quantidade máxima global de produtos: "))

minAmountPerProduct = int(input("Por favor insira a quantidade mínima de cada produto: "))
maxAmountPerProduct  = int(input("Por favor insira a quantidade máxima de cada produto: "))

minProductVariety = int(input("Por favor insira a variedade mínima de produtos: "))
maxProductVariety  = int(input("Por favor insira a variedade máxima de produtos: "))

storeDimensions  = int(input("Por favor insira as dimensões úteis da loja (i.e. volume): "))

In [9]:


# Alternativamente, configuramos as restrições sem input de utilizador para efeitos de análise
minAmountAllProducts = 10
maxAmountAllProducts = 10000
minAmountPerProduct = 0
maxAmountPerProduct  = 1000
minProductVariety = 0
maxProductVariety  = 1000
storeDimensions = 1000000



In [10]:


# Calcula as receitas previstas de um produto tendo em consideração a quantidade escolhida e a quantidade prevista
def getPredictedProfit(productIndex, chosenQuantity):
    product = df.iloc[productIndex]
    if product.periodicity_total_pred_sales == 0:
        return 0
    return min(product.periodicity_total_pred_revenue, (product.periodicity_total_pred_revenue / product.periodicity_total_pred_sales) * chosenQuantity)

# Verificamos as receitas previstas para o primeiro produto, tendo em conta vários valores diferentes, sendo que deverá limitar as receitas consoante as unidades vendidas previstas
print(f"Unidades Vendidas Previstas do Produto {df.iloc[0].product_id}: {df.iloc[0].periodicity_total_pred_sales}")
print(f"Receitas Previstas para 5 unidades escolhidas: {getPredictedProfit(0, 5)}")
print(f"Receitas Previstas para 10 unidades escolhidas: {getPredictedProfit(0, 10)}")
print(f"Receitas Previstas para 15 unidades escolhidas: {getPredictedProfit(0, 15)}")
print(f"Receitas Previstas para 20 unidades escolhidas: {getPredictedProfit(0, 20)}")
print(f"Receitas Previstas para 25 unidades escolhidas: {getPredictedProfit(0, 25)}")




Unidades Vendidas Previstas do Produto P0001: 20
Receitas Previstas para 5 unidades escolhidas: 49.215
Receitas Previstas para 10 unidades escolhidas: 98.43
Receitas Previstas para 15 unidades escolhidas: 147.645
Receitas Previstas para 20 unidades escolhidas: 196.86
Receitas Previstas para 25 unidades escolhidas: 196.86


In [11]:


# Definimos as variáveis de decisão e auxiliares
def getVariables():
    chosenQuantityVars = [] # Inteira, decisão (Armazenará as quantidades escolhidas/ótimas por produto)
    isUsedVars = [] # Binária, auxiliar (Armazenará o valor 1 ou 0, caso o produto seja utilizado ou não, respetivamente)

    for product_id in df.product_id:
        # Aqui, aplicamos já as restrições mínimas e máximas de quantidades individuais, por produto
        chosenQuantityVars.append({product_id:  pulp.LpVariable(product_id, lowBound=minAmountPerProduct, upBound=maxAmountPerProduct,cat='Integer')})
        isUsedVars.append({product_id:  LpVariable(f'isUsed_{product_id}', cat='Binary')})
    
    return chosenQuantityVars, isUsedVars


# Verificamos o resultado para as configurações atuais
chosenQuantityVars, isUsedVars = getVariables()
print(len(chosenQuantityVars))
print(len(isUsedVars))



599
599


In [12]:

# Calcula as receitas previstas por unidade para um dado produto
def getExpectedRevenuePerUnit(product):
    expectedRevenuePerUnit = 0
    if product.periodicity_total_pred_sales > 0:
        expectedRevenuePerUnit = product.periodicity_total_pred_revenue / product.periodicity_total_pred_sales
    return expectedRevenuePerUnit

# Função objetivo
def getObjectiveFunction(decisionVars):
    model = 0
    for index, row in df.iterrows():
        model += (getExpectedRevenuePerUnit(row) * decisionVars[index][row.product_id])
    return model

# Criação do modelo ILP
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)

# Adicionamos a função objetivo ao modelo
chosenQuantityVars, _ = getVariables()
model += getObjectiveFunction(chosenQuantityVars)

# Verificamos o modelo após adição da função objetivo
print(model)



ILP_Problem:
MAXIMIZE
9.843*P0001 + 230.01*P0002 + 0.85*P0004 + 41.660833333333336*P0005 + 10.0*P0007 + 5.385641025641026*P0008 + 5.394651162790698*P0009 + 108.86538461538461*P0012 + 4.641827411167513*P0015 + 1.7653571428571428*P0016 + 4.422730263157895*P0017 + 2.6721343873517784*P0018 + 0.01*P0021 + 29.0*P0022 + 35.275*P0025 + 5.636894736842105*P0026 + 29.050625*P0028 + 15.026666666666666*P0031 + 12.003333333333332*P0033 + 5.062861538461538*P0035 + 4.856181818181818*P0036 + 15.36734693877551*P0038 + 31.36976470588235*P0039 + 3.8814213197969543*P0042 + 24.58*P0043 + 6.6772413793103444*P0044 + 5.028095238095238*P0045 + 10.0*P0047 + 66.25*P0048 + 8.500714285714286*P0049 + 7.25*P0050 + 0.6300641025641026*P0051 + 17.52*P0053 + 7.822211538461539*P0054 + 5.689071428571428*P0055 + 5.764*P0057 + 0.85*P0058 + 15.124125000000001*P0059 + 20.631666666666668*P0060 + 20.43483695652174*P0061 + 5.633448275862069*P0063 + 50.01*P0065 + 0.76*P0066 + 24.41339285714286*P0067 + 4.18*P0068 + 8.24454545454545

In [13]:


# Função que aplica todas as restrições pretendidas para o modelo de otimização
def applyRestrictionsToModel(model, chosenQuantityVars, isUsedVars):
    # Aplicamos a restrição de quantidades globais mínimas e máximas dos produtos
    model += pulp.lpSum([chosenQuantityVars[index][row.product_id] for index, row in df.iterrows()]) <= maxAmountAllProducts
    model += pulp.lpSum([chosenQuantityVars[index][row.product_id] for index, row in df.iterrows()]) >= minAmountAllProducts

    # Aplicamos a restrição de dimensões globais máximas dos produtos, sendo que não deverão exceder as dimensões da loja
    model += pulp.lpSum([row.dimensions*chosenQuantityVars[index][row.product_id] for index, row in df.iterrows()]) <= storeDimensions

    # Aplicamos uma nova restrição de quantidades individuais máximas, consoante a previsão de unidades vendidas de cada produto, 
    # dado que todas as quantidades escolhidas que excedam esta previsão deixam de ser contabilizadas para as receitas e, portanto, devem ser descartadas.
    for index, row in df.iterrows():
        model += chosenQuantityVars[index][row.product_id] <= row.periodicity_total_pred_sales

    # Aplicamos a restrição de variedade globais mínimas e máximas dos produtos, sendo que para tal, iremos precisar de utilizar uma variável binária auxiliar para cada produto
    # que determine se o mesmo está a ser utilizado, isto é, se a sua quantidade escolhida for superior a 0
    for index, row in df.iterrows():
        model += (isUsedVars[index][row.product_id] * row.periodicity_total_pred_sales) >= chosenQuantityVars[index][row.product_id]
    model += pulp.lpSum([isUsedVars[index][row.product_id] for index, row in df.iterrows()]) <= maxProductVariety
    model += pulp.lpSum([isUsedVars[index][row.product_id] for index, row in df.iterrows()]) >= minProductVariety

# Verificamos as restrições
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)
chosenQuantityVars, isUsedVars = getVariables()
model += getObjectiveFunction(chosenQuantityVars)
applyRestrictionsToModel(model, chosenQuantityVars, isUsedVars)
print(model)



ILP_Problem:
MAXIMIZE
9.843*P0001 + 230.01*P0002 + 0.85*P0004 + 41.660833333333336*P0005 + 10.0*P0007 + 5.385641025641026*P0008 + 5.394651162790698*P0009 + 108.86538461538461*P0012 + 4.641827411167513*P0015 + 1.7653571428571428*P0016 + 4.422730263157895*P0017 + 2.6721343873517784*P0018 + 0.01*P0021 + 29.0*P0022 + 35.275*P0025 + 5.636894736842105*P0026 + 29.050625*P0028 + 15.026666666666666*P0031 + 12.003333333333332*P0033 + 5.062861538461538*P0035 + 4.856181818181818*P0036 + 15.36734693877551*P0038 + 31.36976470588235*P0039 + 3.8814213197969543*P0042 + 24.58*P0043 + 6.6772413793103444*P0044 + 5.028095238095238*P0045 + 10.0*P0047 + 66.25*P0048 + 8.500714285714286*P0049 + 7.25*P0050 + 0.6300641025641026*P0051 + 17.52*P0053 + 7.822211538461539*P0054 + 5.689071428571428*P0055 + 5.764*P0057 + 0.85*P0058 + 15.124125000000001*P0059 + 20.631666666666668*P0060 + 20.43483695652174*P0061 + 5.633448275862069*P0063 + 50.01*P0065 + 0.76*P0066 + 24.41339285714286*P0067 + 4.18*P0068 + 8.24454545454545

In [14]:


# Função para executar o modelo de otimização
def runILP():
    model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)
    chosenQuantityVars, isUsedVars = getVariables()
    model += getObjectiveFunction(chosenQuantityVars)
    applyRestrictionsToModel(model, chosenQuantityVars, isUsedVars)
    status = model.solve()
    return model, status, chosenQuantityVars

# Executamos o modelo de otimização
model, status, chosenQuantityVars = runILP()
# Verificamos o sucesso do modelo, isto é, se conseguiu obter uma solução ótima
print(f"Resultado do Modelo: {pulp.LpStatus[status]}")

# Verificamos a solução obtida pelo modelo, isto é, as receitas previstas para as quantidades escolhidas
print(f"\nTotal de Receitas Previstas: \n{round(pulp.value(model.objective), 2)}$")



Resultado do Modelo: Optimal

Total de Receitas Previstas: 
82408.19$


In [15]:


# Criamos um dataframe onde iremos armazenar os resultados deste modelo, para futuramente comparar com os resultados obtidos no algoritmo genético
model_ilp_results = pd.DataFrame(columns=["product_id", "quantidadeOtima"])

# Verificamos as quantidades escolhidas pelo modelo para cada produto
for index, row in df.iterrows():
    model_ilp_results = pd.concat([model_ilp_results,  pd.DataFrame.from_records([{"product_id": row.product_id, "quantidadeOtima": pulp.value(chosenQuantityVars[index][row.product_id])}])])

print(model_ilp_results.head(25))


  product_id  quantidadeOtima
0      P0001              0.0
0      P0002              0.0
0      P0004              0.0
0      P0005              0.0
0      P0006              0.0
0      P0007              0.0
0      P0008              0.0
0      P0009              0.0
0      P0010              0.0
0      P0011              0.0
0      P0012              0.0
0      P0014              0.0
0      P0015              0.0
0      P0016              0.0
0      P0017              0.0
0      P0018              0.0
0      P0021              0.0
0      P0022              0.0
0      P0024              0.0
0      P0025              0.0
0      P0026              0.0
0      P0028             16.0
0      P0029              0.0
0      P0031              3.0
0      P0033              0.0


In [16]:

df2 = df.copy()
df = df.head(25)

----------- Genetic Algorithm -------------

    Restritions:
    

Restrictions functions return 1 respected, 0 if not respected


In [17]:


# Validação das quantidades escolhidas de um indíviduo perante as restrições de quantidades globais máximas e mínimas
# Caso o indíviduo esteja válido, retorna 0
# Caso o indíviduo esteja inválido, retorna a maior diferença absoluta
def validateGlobalAmounts(individuo):
    somaTotalAmountIndividuo = sum(individuo)

    if somaTotalAmountIndividuo < minAmountAllProducts:
        return somaTotalAmountIndividuo - minAmountAllProducts

    if somaTotalAmountIndividuo > maxAmountAllProducts:
        return somaTotalAmountIndividuo - maxAmountAllProducts

    return 0



In [19]:
#Dimension Restriction
def validateDimension(individuo):
    totalDimension = 0
    contador = 0
    for indAux in individuo:
        totalDimension = indAux * df.iloc[contador].dimensions
        contador = contador + 1
    if(totalDimension > storeDimensions):
        return 0
    else:
        return 1

In [20]:


# Returna 0 se for válido
# Return um valor diferente de 0 (+ ou -) consoante a maior diferença absoluta de quantidades globais inválida encontrada 
def validateInd(ind):
    #validate that second child is good
    diffGlobalQuantities = validateGlobalAmounts(ind)

    # totalQuantityRestriction = validateTotalQuantity(ind)
    dimensionRestriction = validateDimension(ind)
    if (diffGlobalQuantities != 0) and (dimensionRestriction == 1):
        return True
    return False

def adjustQuantities(ind, diff_quant):
    firstPaiWithPredictions = pd.DataFrame(df, columns=["periodicity_total_pred_sales",  "periodicity_total_pred_revenue"])
    firstPaiWithPredictions['chosen_quantity'] = ind
    # Armazena a diferença entre a quantidade prevista de vendas e a quantidade escolhida
    # Exemplo: 
    # Quantidade Prevista = 10
    # Quantidade Escolhida = 5
    # Diferença = 10 - 5 = 5
    firstPaiWithPredictions['quantity_diff'] = firstPaiWithPredictions['periodicity_total_pred_sales'] - firstPaiWithPredictions['chosen_quantity']
    
    # Se existirem quantidades globais em excesso então iremos remover
    if diff_quant > 0:
        # Ordenamos os produtos por ordem ascendente dos que têm mais quantidades em excesso (> Quantidades Previstas)
        # e, de seguida, pelos que têm menos receitas previstas dado que iremos remover quantidades
        firstPaiWithPredictions = firstPaiWithPredictions.sort_values(by=['quantity_diff', 'periodicity_total_pred_revenue'], ascending=[True, True])
        indexToAdjust = 0
        while diff_quant > 0:
            quantityToAdjust = abs(firstPaiWithPredictions.iloc[indexToAdjust]['quantity_diff'])
            if quantityToAdjust > diff_quant:
                quantityToAdjust = diff_quant
            ind[indexToAdjust] = quantityToAdjust
            diff_quant -= quantityToAdjust
            indexToAdjust += 1
    # Se existirem quantidades globais em falta então iremos adicionar
    if diff_quant < 0:
        # Ordenamos os produtos por ordem descendente dos que têm mais quantidades em falta (< Quantidades Previstas) 
        # e, de seguida, pelos que têm mais receitas previstas dado que iremos adicionar quantidades
        firstPaiWithPredictions = firstPaiWithPredictions.sort_values(by=['quantity_diff', 'periodicity_total_pred_revenue'], ascending=[False, False])
        indexToAdjust = 0
        while diff_quant < 0:
            quantityToAdjust = abs(firstPaiWithPredictions.iloc[indexToAdjust]['quantity_diff'])
            if quantityToAdjust > abs(diff_quant):
                quantityToAdjust = abs(diff_quant)
            ind[indexToAdjust] += quantityToAdjust
            diff_quant += quantityToAdjust
            indexToAdjust += 1
    
    return ind

Creation of First Population

In [21]:
#Create first population 


#parameters entry:
    #list population []
    #dictionary products [{'productName': 'p1', 'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5},...]
    #populationSize ex:2
#return: list of products with theirs quantities [[3, 3, 3, 1], [3, 1, 3, 3]]
def createFirstPopulation(populationSize):
    pop = []

    while len(pop) < populationSize:
        solution = []
        totalQuantity = 0
        totalDimension = 0
        
        #generates the chromossomes each representing one product
        for j in range(len(df)):
            quantity = random.randint(minAmountPerProduct, round((maxAmountPerProduct / 3)))
            solution.append(quantity)
        #     print("quantity",quantity)
        # print("solution",solution)
        # print("antesValidacap")
        #Total Amount Restriction
        totalAmountRestriction = validateGlobalAmounts(solution)
        # print("validateGlobalAmounts",totalAmountRestriction)


        
        # diff_quant = validateGlobalAmounts(solution) 

        # while diff_quant != 0:
        #     adjustQuantities(solution, diff_quant)
        #     diff_quant = validateGlobalAmounts(solution)

        #Total Quantity Restriction
        totalQuantityRestriction = validateTotalQuantity(solution)
        print("totalQuantityRestriction",totalQuantityRestriction)
        #Dimension Restriction
        dimensionRestriction = validateDimension(solution)
        print("dimensionRestriction",dimensionRestriction)
        
        # if (totalAmountRestriction == 0) and
        if (totalQuantityRestriction == 1) and (dimensionRestriction == 1):
            pop.append(solution)
        
    return pop

print(createFirstPopulation(1))

somaTotalAmountIndividuo 4725
totalQuantityRestriction 1
dimensionRestriction 1
[[275, 274, 96, 185, 308, 32, 315, 190, 192, 267, 6, 160, 323, 206, 74, 166, 112, 289, 109, 15, 195, 280, 222, 213, 221]]


Evaluation Function

In [14]:
# #get profict of product with quantity 
# def getPredictedProfit(product, chosenQuantity):
#     return min(product['predictedProfits'], (product['predictedProfits']/product['predictedProductQuantities']) * chosenQuantity)

# Calcula as receitas previstas de um produto tendo em consideração a quantidade escolhida e a quantidade prevista
def getPredictedProfit(productIndex, chosenQuantity):
    product = df.iloc[productIndex]
    lis= min(product.periodicity_total_pred_revenue, (product.periodicity_total_pred_revenue / product.periodicity_total_pred_sales) * chosenQuantity)

    return lis
# Testamos a função
print(getPredictedProfit(0, 5))

49.215


In [440]:
print(getPredictedProfit(0, 5))

49.215


In [15]:

#Evaluates individuos by their profict

#parameters entry:
    #list population with values: [[1,2,3,4],[1,7,5,4]]
    #dictionary products [{'productName': 'p1', 'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5},...]
    #listPopulationWithProfit []
    
# returns dictionary with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
def createFitnessFunction(population):

    listPopulationWithIndiviuoAndProfit=[]
    #population= [[1,2,3,4],[1,7,5,4]]
    for individuo in population:
        individuoProfit = 0
        contador = 0
        #individuo= [1,2,3,4]
        for productQuantity in individuo:
   
            profitProduct=getPredictedProfit(contador, productQuantity)
            individuoProfit = individuoProfit + profitProduct
            listPopulationWithProfitAux = {'individuo': individuo, 'calculatedProfit':individuoProfit}
            contador = contador + 1

            
        listPopulationWithIndiviuoAndProfit.append(listPopulationWithProfitAux)


    return listPopulationWithIndiviuoAndProfit

In [443]:
listPopulationWithProfit=createFirstPopulation(10)
listPopulationWithIndiviuoAndProfit=createFitnessFunction(listPopulationWithProfit,)

print(listPopulationWithIndiviuoAndProfit)

KeyboardInterrupt: 

Crossover

In [16]:
#Evaluates individuos by their profict

#parameters entry:
    #listPopulationWithIndiviuoAndProfit with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    
# returns:
    # listPopulationWithProfitOrdered same as listPopulationWithIndiviuoAndProfit but Ordered  
    # populationOrderedByProfit population ordered [[3, 2, 2, 3],..]
def getListsOrderByProfit(listPopulationWithIndiviuoAndProfit):
    listPopulationWithProfitOrdered=sorted(listPopulationWithIndiviuoAndProfit,key=lambda x: x['calculatedProfit'], reverse=True)
    populationOrderedByProfit=[d['individuo'] for d in listPopulationWithProfitOrdered]
    # print(listPopulationWithProfitOrdered)
    # print(populationOrderedByProfit)
    return listPopulationWithProfitOrdered,populationOrderedByProfit

In [115]:
listPopulationWithProfitOrdered, populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
print(listPopulationWithProfitOrdered)
print(populationOrderedByProfit)

[{'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [2, 3, 3, 2], 'calculatedProfit': 90.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [2, 3, 3, 3], 'calculatedProfit': 90.0}, {'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 1, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [2, 2, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}]
[[3, 3, 3, 3], [3, 3, 3, 3], [2, 3, 3, 2], [3, 3, 2, 3], [3, 3, 2, 3], [2, 3, 3, 3], [3, 2, 2, 3], [3, 1, 3, 3], [2, 2, 3, 3], [3, 2, 2, 3]]


In [17]:

#parameters entry:
    #list initialPopulation with values: [[1,2,3,4],[1,7,5,4]] already order by profi
    #list listPopulationGoingToCrossover: []
    #crossoverTax: tax to calculate numer of elements to do crossover ex: 0.10-> 90% do crossover
    #listPopulationWithIndiviuoAndProfit: dictionary with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    
# returns finalPopulation- list that does not  : [[1,2,3,4],[1,7,5,4]] that will then go to mutation
def getCrossoverListAccordingToCrossoverTax(initialPopulation,crossoverTax):
    print("initialPopulation",initialPopulation)
    numberElementsNoCrossover = round(len(initialPopulation)*crossoverTax)

    #EX:[[1,2,3,4],[1,7,5,4]]
    finalPopulation = [individuo for individuo in initialPopulation[:numberElementsNoCrossover]]
    #EX:[[1,2,3,4],[1,7,5,4]
    listCrossOver = [individuo for individuo in initialPopulation[numberElementsNoCrossover+1:]]

    return finalPopulation,listCrossOver

In [121]:
listPopulationWithProfitOrdered, populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
finalPopulation,listCrossOver=getCrossoverListAccordingToCrossoverTax(populationOrderedByProfit,0.10)
print(finalPopulation)
print(listCrossOver)




[[3, 3, 3, 3]]
[[2, 3, 3, 2], [3, 3, 2, 3], [3, 3, 2, 3], [2, 3, 3, 3], [3, 2, 2, 3], [3, 1, 3, 3], [2, 2, 3, 3], [3, 2, 2, 3]]


In [18]:
#parameters entry:
    #listToCrossOver: [[1,2,3,4],[1,7,5,4]] already order by profi

# returns listpopulationAfterCrossover [[1,2,3,4],[1,7,5,4]] already changed after crossover
def crossoverFunction(listToCrossOver):

    listpopulationAfterCrossover=[]

    while len(listToCrossOver)>0:
    
        if len(listToCrossOver) == 1:
            #listToCrossOver[0]= last individuo remaining [1,2,3,4]
            listpopulationAfterCrossover.append(listToCrossOver[0])
            listToCrossOver.remove(listToCrossOver[0])
            
        else:
            
            #generates random numbers to discover wich individuos ae going to crossover
            indexFirstPai = random.randint(0, len(listToCrossOver)-1)
            indexSecondPai = random.randint(0, len(listToCrossOver)-1)

            #garantees that is not selected the same individu
            while indexFirstPai == indexSecondPai:
                indexSecondPai = random.randint(0, len(listToCrossOver)-1)
            
            #individuo to crossover
            firstPai = listToCrossOver[indexFirstPai]
            secondPai = listToCrossOver[indexSecondPai]
            
            # firstPaiToDelete = listToCrossOver[indexFirstPai].copy()
            # secondPaiToDelete = listToCrossOver[indexSecondPai].copy()
            crossoverPoint = random.randint(1, len(listToCrossOver[indexFirstPai])-1)
            
            for i in list(range(crossoverPoint)):
                aux = firstPai[i]
                firstPai[i] = secondPai[i]
                secondPai[i] = aux

            diff_quant = validateGlobalAmounts(firstPai) 
            while diff_quant != 0:
                adjustQuantities(firstPai, diff_quant)
                diff_quant = validateGlobalAmounts(firstPai)

            while diff_quant != 0:
                adjustQuantities(secondPai, diff_quant)
                diff_quant = validateGlobalAmounts(secondPai)

            #validate that first child is good
            # totalAmountRestriction1 = validateGlobalAmounts(firstPai)
            # totalQuantityRestriction1 = validateTotalQuantity(firstPai)
            # dimensionRestriction1 = validateDimension(firstPai, products)
            # #validate that second child is good
            # totalAmountRestriction2 = validateGlobalAmounts(secondPai)
            # totalQuantityRestriction2 = validateTotalQuantity(secondPai)
            # dimensionRestriction2 = validateDimension(secondPai, products)

            # if (totalAmountRestriction1 == 1) and (totalQuantityRestriction1 == 1) and (dimensionRestriction1 == 1) and (totalAmountRestriction2 == 1) and (totalQuantityRestriction2 == 1) and (dimensionRestriction2 == 1):
                
            listpopulationAfterCrossover.append(firstPai)
            listpopulationAfterCrossover.append(secondPai)
            listToCrossOver.pop(max(indexFirstPai,indexSecondPai))
            listToCrossOver.pop(min(indexFirstPai,indexSecondPai))
           

  

    return listpopulationAfterCrossover





In [281]:

print(list(range(1)))

[0]


In [273]:


listPopulationWithProfitOrdered, populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
finalPopulation,listCrossOver=getCrossoverListAccordingToCrossoverTax(populationOrderedByProfit,0.10)

print(crossoverFunction(listCrossOver))

KeyboardInterrupt: 

In [263]:
print(minAmountPerProduct)
print(maxAmountPerProduct)
print(minAmountAllProducts)
print(maxAmountAllProducts)

10
1000
100
10000


In [23]:
#parameters entry:
    #list population with values: [[1,2,3,4],[1,7,5,4]]
    #crossoverTax: tax to calculate numer of elements to do crossover ex: 0.10-> 90% do crossover
    #listPopulationWithIndiviuoAndProfit: dictionary with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    
# returns finalPopulation : [[1,2,3,4],[1,7,5,4]] that will then go to mutation and population ordered [[1,2,3,4],[1,7,5,4]]
def crossover(listPopulationWithIndiviuoAndProfit,crossoverTax):
    print("Mutacaoaaaaaaaaaaaaaaaaaa",listPopulationWithIndiviuoAndProfit)
    #function that returns dic ordered [{'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, and 
    listPopulationWithProfitOrdered,populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
    print("Mutacaoaaaaaaaaaaaaaaaaaa2",populationOrderedByProfit)
    #list with population (size according to crossoverTax) after crossover
    listCrossover=[]
   
    #function that returns final list of population that doesn't go to crossover[[1,2,3,4],[1,7,5,4]] and list that goes to crossover[[1,2,3,4],[1,7,5,4]]
    finalPopulation,listToCrossOver=getCrossoverListAccordingToCrossoverTax(populationOrderedByProfit,crossoverTax)

    #crossover function that receives list listToCrossOver and returns list after crossover [[1,2,3,4],[1,7,5,4]]
    listpopulationAfterCrossover=crossoverFunction(listToCrossOver)
    
    #population with individuos after crossover being added to the finalpopulation list that is going to go to mutation (size=initialSize)
    finalPopulation+=listpopulationAfterCrossover
    print(finalPopulation)

    return finalPopulation

# print(maxAmountPerProduct)
# print(minAmountPerProduct)
# print(maxAmountAllProducts)
# print(minAmountAllProducts)
listPopulationWithProfit=createFirstPopulation(10)
print(listPopulationWithProfit)
listPopulationWithIndiviuoAndProfit=createFitnessFunction(listPopulationWithProfit)
print("listPopulationWithIndiviuoAndProfit",listPopulationWithIndiviuoAndProfit)
print("finalPopulation",crossover(listPopulationWithIndiviuoAndProfit,0.1))




somaTotalAmountIndividuo 100614
indAux 102
1
1000
indAux 218
1
1000
indAux 236
1
1000
indAux 327
1
1000
indAux 78
1
1000
indAux 15
1
1000
indAux 119
1
1000
indAux 7
1
1000
indAux 209
1
1000
indAux 121
1
1000
indAux 121
1
1000
indAux 190
1
1000
indAux 286
1
1000
indAux 124
1
1000
indAux 296
1
1000
indAux 272
1
1000
indAux 223
1
1000
indAux 25
1
1000
indAux 257
1
1000
indAux 113
1
1000
indAux 146
1
1000
indAux 203
1
1000
indAux 308
1
1000
indAux 134
1
1000
indAux 93
1
1000
indAux 131
1
1000
indAux 227
1
1000
indAux 216
1
1000
indAux 291
1
1000
indAux 321
1
1000
indAux 226
1
1000
indAux 38
1
1000
indAux 147
1
1000
indAux 258
1
1000
indAux 173
1
1000
indAux 181
1
1000
indAux 74
1
1000
indAux 171
1
1000
indAux 28
1
1000
indAux 181
1
1000
indAux 16
1
1000
indAux 61
1
1000
indAux 282
1
1000
indAux 258
1
1000
indAux 92
1
1000
indAux 201
1
1000
indAux 314
1
1000
indAux 311
1
1000
indAux 278
1
1000
indAux 95
1
1000
indAux 330
1
1000
indAux 52
1
1000
indAux 88
1
1000
indAux 202
1
1000
indAux 142


C:\Users\Isa\AppData\Local\Temp\ipykernel_12104\2537956609.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  lis= min(product.periodicity_total_pred_revenue, (product.periodicity_total_pred_revenue / product.periodicity_total_pred_sales) * chosenQuantity)
C:\Users\Isa\AppData\Local\Temp\ipykernel_12104\2537956609.py:8: RuntimeWarning: invalid value encountered in double_scalars
  lis= min(product.periodicity_total_pred_revenue, (product.periodicity_total_pred_revenue / product.periodicity_total_pred_sales) * chosenQuantity)


listPopulationWithIndiviuoAndProfit [{'individuo': [102, 218, 236, 327, 78, 15, 119, 7, 209, 121, 121, 190, 286, 124, 296, 272, 223, 25, 257, 113, 146, 203, 308, 134, 93, 131, 227, 216, 291, 321, 226, 38, 147, 258, 173, 181, 74, 171, 28, 181, 16, 61, 282, 258, 92, 201, 314, 311, 278, 95, 330, 52, 88, 202, 142, 289, 142, 213, 296, 71, 230, 215, 153, 244, 188, 316, 241, 93, 224, 146, 193, 139, 264, 60, 144, 198, 65, 326, 212, 70, 129, 46, 205, 49, 177, 20, 15, 6, 79, 323, 268, 67, 325, 137, 79, 327, 108, 148, 242, 237, 220, 125, 252, 188, 27, 215, 123, 225, 31, 137, 259, 47, 26, 11, 161, 266, 237, 330, 182, 323, 222, 85, 236, 165, 34, 152, 238, 122, 204, 252, 118, 233, 3, 20, 118, 137, 324, 13, 330, 223, 113, 252, 42, 276, 123, 49, 238, 148, 205, 313, 241, 219, 128, 295, 306, 57, 234, 143, 112, 219, 15, 85, 86, 72, 280, 17, 284, 194, 304, 204, 174, 299, 184, 118, 143, 222, 96, 57, 127, 290, 51, 190, 118, 249, 97, 333, 175, 324, 52, 291, 151, 235, 103, 260, 112, 145, 148, 302, 17, 268, 30

Mutation

In [301]:
#parameters entry:
    #list population with values after crossover: [[1,2,3,4],[1,7,5,4]]
    #taxMutation tax to calculate number of elements to do crossover ex: 5-> 5% do mutation
    
# returns listPopulationToMutate : [[1,2,3,4],[1,7,5,4]] after mutation
def getMutationListAccordingToMutationTax(finalPopulation,taxMutation):
    mutationList=finalPopulation[:]
    for x in mutationList:
        #generates random number 
        t=random.randint(0, 100)

        #when random number if > taxMutation then removes values from list, because this is has high probability the list of to mutate is going to be small has expected
        if t >= taxMutation:
            mutationList.remove(x)
    return mutationList

In [388]:
def mutationFunction(finalPopulation,listPopulationToMutate,taxChangeGene):
    print("finalPopulation",finalPopulation)
    for individuo in finalPopulation:
    
            initialEl=individuo.copy()
            
            for l in listPopulationToMutate:
                
                if individuo==l:
                    
                        print("individuoMUTATIONFfunction",individuo)
                        print("quantiIN",individuo)
                        print("LENNNNNN individuo",len(individuo))
                        index=random.randint(0, len(individuo)-1)
                        print("idndex",index)
                        for i in list(range(len(individuo))):
                            randomSumOrLess = random.randint(0, 1)
                            
                            if(randomSumOrLess == 1):
                                print("localMutation",individuo[index])
                                individuo[index] = individuo[index] + round(individuo[index]*taxChangeGene)
                            else:
                                individuo[index] = individuo[index] - round(individuo[index]*taxChangeGene)
                            
                                    
                            totalAmountRestriction = validateGlobalAmounts(individuo)
                            totalQuantityRestriction = validateTotalQuantity(individuo)
                            dimensionRestriction = validateDimension(individuo)
                                    
                            if (totalAmountRestriction == 0) or (totalQuantityRestriction != 0) or (dimensionRestriction == 0):
                                individuo=initialEl
                        
    return finalPopulation

In [383]:
#parameters entry:
    #list population with values after crossover: [[1,2,3,4],[1,7,5,4]]
    #taxMutation tax to calculate number of elements to do crossover ex: 5-> 5% do mutation
    #changeGene: tax to calculate how the quantity of a product is going to change, ex: 0.1 QP1=2 QP1=2*2*0.1
    
# returns finalPopulation : [[1,2,3,4],[1,7,5,4]] after mutation
def mutation(finalPopulation, taxMutation, taxChangeGene):

    #function that returns final list of population that doesn't go to crossover[[1,2,3,4],[1,7,5,4]] and list that goes to crossover[[1,2,3,4],[1,7,5,4]]
    listPopulationToMutate=getMutationListAccordingToMutationTax(finalPopulation,taxMutation)

    #mutation function that receives list listPopulationToMutate and returns list after mutation [[1,2,3,4],[1,7,5,4]]=Final population to go then to next generation
    listpopulationAfterMutation=mutationFunction(finalPopulation,listPopulationToMutate,taxChangeGene)
    
    return listpopulationAfterMutation

Iterate multiple times

In [421]:
from builtins import list
previousPop=createFirstPopulation(2)
#parameters entry:
    #list population with values after crossover: [[1,2,3,4],[1,7,5,4]]
    #taxMutation tax to calculate number of elements to do crossover ex: 5-> 5% do mutation
    #changeGene: tax to calculate how the quantity of a product is going to change, ex: 0.1 QP1=2 QP1=2*2*0.1
    
# returns finalPopulation : [[1,2,3,4],[1,7,5,4]] after mutation
def generatePopulation(previousPop, crossoverTax, taxMutation, taxChangeGene):
    #creates dic with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    listPopulationWithIndiviuoAndProfit=createFitnessFunction(previousPop)
    print("1Pop",listPopulationWithIndiviuoAndProfit)
    
    #returns listpopulationAfterCrossover [[1,2,3,4],[1,7,5,4]] already changed after crossover
    listpopulationAfterCrossover=crossover(listPopulationWithIndiviuoAndProfit,crossoverTax)

    #returns finalPopulation : [[1,2,3,4],[1,7,5,4]] after mutation
    listAfterMutation=mutation(listpopulationAfterCrossover, taxMutation, taxChangeGene)

    return listAfterMutation
    print("--listAfterMutation--",listAfterMutation)


    # listPopulationWithProfit=createFirstPopulation(2)
# listPopulationWithIndiviuoAndProfit=createFitnessFunction(listPopulationWithProfit)
# print("listPopulationWithIndiviuoAndProfit",listPopulationWithIndiviuoAndProfit)
# print("finalPopulation",crossover(listPopulationWithIndiviuoAndProfit,0.1))

print("POPULACAO",generatePopulation(previousPop,0.1,5,0.1))

1Pop [{'individuo': [194, 21, 330, 150], 'calculatedProfit': 927.6500000000001}, {'individuo': [230, 58, 114, 102], 'calculatedProfit': 927.6500000000001}]
Mutacaoaaaaaaaaaaaaaaaaaa [{'individuo': [194, 21, 330, 150], 'calculatedProfit': 927.6500000000001}, {'individuo': [230, 58, 114, 102], 'calculatedProfit': 927.6500000000001}]
Mutacaoaaaaaaaaaaaaaaaaaa2 [[194, 21, 330, 150], [230, 58, 114, 102]]
[[230, 58, 114, 102]]
finalPopulation [[230, 58, 114, 102]]
POPULACAO [[230, 58, 114, 102]]


In [414]:
def runGeneticAlgorithm(popSize, nIterations, crossoverTax, taxMutation, taxChangeGene):
    start_time = time.time()
    it = 0
    popsList = []
    
    
    while it <= nIterations:
        if it == 0:
            #creates first population with random quantities
            firstPop=createFirstPopulation(popSize)
            #popsList+=firstPop
            print("1x",firstPop)
        else:
            newPop = generatePopulation(firstPop,crossoverTax, taxMutation, taxChangeGene)
            print("newPop",newPop)
            popsList+=newPop
        it += 1
    print("----------------------------------------------")  
    print(popsList)   
nIterations = random.randint(0, 100)
print(nIterations)

popSize = random.randint(1,100)
print(popSize)

crossoverTax = round(random.uniform(0.01, 0.10),2)
print(crossoverTax)

taxMutation = random.randint(1,5)
print(taxMutation)

taxChangeGene = round(random.uniform(0.01,0.1),2)
print(taxChangeGene)

#runGeneticAlgorithm(popSize, nIterations, crossoverTax, taxMutation, taxChangeGene)
runGeneticAlgorithm(2, 2, 0.1, 5, 0.1)




87
64
0.05
2
0.06
1x [[143, 192, 71, 27], [112, 268, 50, 82]]
[{'individuo': [143, 192, 71, 27], 'calculatedProfit': 927.6500000000001}, {'individuo': [112, 268, 50, 82], 'calculatedProfit': 927.6500000000001}]
[[112, 268, 50, 82]]
finalPopulation [[112, 268, 50, 82]]
newPop [[112, 268, 50, 82]]
[{'individuo': [143, 192, 71, 27], 'calculatedProfit': 927.6500000000001}, {'individuo': [112, 268, 50, 82], 'calculatedProfit': 927.6500000000001}]
[[112, 268, 50, 82]]
finalPopulation [[112, 268, 50, 82]]
newPop [[112, 268, 50, 82]]
----------------------------------------------
[[112, 268, 50, 82], [112, 268, 50, 82]]


In [19]:
# def time_plot(valuesListToCompare, label):
#     fig, ax = plt.subplots(figsize=(15,5))
#     sns.lineplot(x = values1.index.values, y = values1, data=data, ax=ax, color='blue', label=label)
#     sns.lineplot(x = values2.index.values, y = values2, data=data, ax=ax, color='red', label=label)
#     sns.lineplot(x = values3.index.values, y = values3, data=data, ax=ax, color='orange', label=label)
#     sns.lineplot(x = values4.index.values, y = values3, data=data, ax=ax, color='pink', label=label)



# values1 =  geneticAlgorithm(products, populationSize,crossoverTax,taxMutation, 0.7)
# values2 =  geneticAlgorithm(products, populationSize,crossoverTax,taxMutation, 0.8)
# values3 =  geneticAlgorithm(products, populationSize,crossoverTax,taxMutation, 0.5)
# values4 =  geneticAlgorithm(products, populationSize,crossoverTax,taxMutation, 0.6)

# time_plot(values1)

NameError: name 'geneticAlgorithm' is not defined

In [17]:
 = 

SyntaxError: invalid syntax (2304172042.py, line 1)

In [ ]:
def iterate():

    iterations = random.randint(0, 100)
    print(iterations)

    populationSize = random.randint(1,100)
    print(populationSize)

    contador = 0
    while contador < iterations:

        print(populationSize)
        crossoverTax = round(random.uniform(0.01, 0.10),2)
        print(crossoverTax)
        taxMutation = random.randint(1,5)
        print(taxMutation)
        taxChangeGene = round(random.uniform(0.01,0.1),2)
        print(taxChangeGene)

 
        geneticAlgorithm(products, populationSize,crossoverTax,taxMutation,taxChangeGene)
